In [2]:
import json
import os
import pandas as pd
import pickle # to create file with msg
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [3]:
PATH = os.getcwd()
dataset_path = os.path.join(PATH, "data/")

In [4]:
json_file = open(dataset_path+'database_terrapins.json')
data = json.load(json_file)

In [5]:
data = data['objects']

In [6]:
messages = [tx_1['tx']['value']['msg'] for tx_1 in data]

In [7]:
N = len(messages)
new_m = []
y =[]
for i in range(0, N):
    if 'execute_msg' in [m['value'].keys() for m in messages[i]][0]:
        new_m.append(messages[i])
    else:
        del data[i]
    

In [8]:
file_to_write = open(dataset_path+"messeges.pickle", "ab")
counter = 0
for tx_i in data:
    messages_tx_i = {}
    messages_i = tx_i['tx']['value']['msg']
    timestamp = tx_i['timestamp']
    hash_id = tx_i['txhash']
    for msg in messages_i:
        if 'execute_msg' in msg['value'].keys():
            typeInfo = msg['type']
            exe_msg = msg['value']['execute_msg']
            dictObj = {'type' : typeInfo, 'hash_id': hash_id, 'exe_msg':exe_msg, 'timestamp':timestamp}
            pickle.dump(dictObj, file_to_write)
            counter+=1
file_to_write.close()

In [9]:
counter = 135820

In [10]:
# unpickling file
with open(dataset_path+'messeges.pickle', 'rb') as handle:
    all_messages = [pickle.load(handle) for k in range(counter)] # 135820 to jest liczba wszystkich transakcji, które się nadawają == counter 
handle.close()

In [11]:
mint_nfts = []
with open(dataset_path+"mint_nfts.pickle", "ab") as file:
    for msg in all_messages:
        if "mint_nft" in msg['exe_msg'].keys():
            mint_nfts.append(msg)
            pickle.dump(msg, file)

In [12]:
with open(dataset_path+'mint_nfts.pickle', 'rb') as f:
    count = 10000
    mint_nfts = [pickle.load(f) for i in range(count)]
f.close()

In [13]:
exe = []
for mint in all_messages:
    if 'execute_order' in mint['exe_msg'] :
        exe.append(mint)

In [14]:
hash_id= []
token_id = []
currency = []
amount = []
timestamp = []
for e in exe:
    if 'nft' in e['exe_msg']['execute_order']['order']['order']['maker_asset']['info']:
        token_id.append(e['exe_msg']['execute_order']['order']['order']['maker_asset']['info']['nft']['token_id'])
        currency.append(e['exe_msg']['execute_order']['order']['order']['taker_asset']['info']['native_token']['denom'])
        amount.append(e['exe_msg']['execute_order']['order']['order']['taker_asset']['amount'])
    else:
        token_id. append(e['exe_msg']['execute_order']['order']['order']['taker_asset']['info']['nft']['token_id'])
        currency.append(e['exe_msg']['execute_order']['order']['order']['maker_asset']['info']['native_token']['denom'])
        amount.append(e['exe_msg']['execute_order']['order']['order']['maker_asset']['amount'])
    
    hash_id.append(e['hash_id'])
    timestamp.append(e['timestamp'])

In [15]:
df = pd.DataFrame(list(zip(token_id, hash_id, currency, amount, timestamp)),
            columns = ['token_id', 'hash_id','currency', 'amount', 'timestamp'])
df.to_pickle(dataset_path+'transactions.pkl')

In [16]:
columns = ['Name', 
           'Background', 
           'Skin', 
           'Shell', 
           'Body', 
           'Mouth', 
           'Eyes', 
           'Head', 
           'Token_ID']

In [17]:
mintNFT_df = pd.DataFrame(columns = columns)
mintNFT_df

,Name,Background,Skin,Shell,Body,Mouth,Eyes,Head,Token_ID


In [18]:
i = 0
for mint in mint_nfts:
    name = mint['exe_msg']['mint_nft']['extension']['name']
    token_id = mint['exe_msg']['mint_nft']['token_id']
    attributes = mint['exe_msg']['mint_nft']['extension']['attributes']
    attr_details = []
    for attr in attributes:
        attr_details.append(attr['value'])
    mintNFT_df.loc[i] = [name] + attr_details + [token_id]
    i+=1

In [19]:
mintNFT_df

,Name,Background,Skin,Shell,Body,Mouth,Eyes,Head,Token_ID
0,Terrapin #04880,Mint,Geisha,Knight Sword - Brown,Naked,Snarl - Cigar,Cyborg - Closed Eye,Spiky Hair - Brown,21115668659623778144848103954867683887
1,Terrapin #09271,Mint,Lion,Base Shell,Naked,Snarl,Tweeker Eyes,Mohawk - Black,339686573371205809325137876315247034028
2,Terrapin #00735,Blue Grey,Geisha,Diamond Back,Steam Punk Shirt - Red,Grin - 5pm Shadow,Monocle - Bored Eyes,None,26845686848091010612034591826243939022
3,Terrapin #08213,Lilac,Common,Base Shell,Leather Jacket - Black and White,Dopey,Eyepatch - Bored,Girls Hair - Brown,29725014212231427076011833295205785773
4,Terrapin #03821,Mint,Lion,Base Shell,Leather Jacket - Black and White,Grin - 5pm Shadow,Eyepatch - Angry,Pylon Helmet,208630217786676767860015082049195109707
...,...,...,...,...,...,...,...,...,...
9995,Terrapin #00146,Charcoal Full Moon,Tiger Teddy,Shell Spikes,Naked,Terrorpin,Steam Punk - Tweeker,Spiky Hair - Black,140850908578020633267983016681440335825
9996,Terrapin #00147,Early Light,Common,Base Shell,Hawaiian Shirt - Pineapple,Moustache,Cyborg - Bored,Lion Hat,198469524447758964403111579286542946696
9997,Terrapin #00148,Midnight Blue - Big Moon,Tiger Teddy,Angel Wings,Naked,Dopey,Angry Tweeker Eyes,Lion Hat,7349082768268168848355752264853030978
9998,Terrapin #00149,Mint,Tiger,Base Shell,Naked,Dopey,Eyepatch - Bored,Lion Hat,207748826621535591411786181607822466045


In [20]:
mintNFT_df['Shell'] = mintNFT_df['Shell'].str.replace(' Shell', '')
mintNFT_df['Eyes'] = mintNFT_df['Eyes'].str.replace(' Eyes', '')
mintNFT_df['Head'] = mintNFT_df['Head'].str.replace(' Hair', '')
mintNFT_df

,Name,Background,Skin,Shell,Body,Mouth,Eyes,Head,Token_ID
0,Terrapin #04880,Mint,Geisha,Knight Sword - Brown,Naked,Snarl - Cigar,Cyborg - Closed Eye,Spiky - Brown,21115668659623778144848103954867683887
1,Terrapin #09271,Mint,Lion,Base,Naked,Snarl,Tweeker,Mohawk - Black,339686573371205809325137876315247034028
2,Terrapin #00735,Blue Grey,Geisha,Diamond Back,Steam Punk Shirt - Red,Grin - 5pm Shadow,Monocle - Bored,None,26845686848091010612034591826243939022
3,Terrapin #08213,Lilac,Common,Base,Leather Jacket - Black and White,Dopey,Eyepatch - Bored,Girls - Brown,29725014212231427076011833295205785773
4,Terrapin #03821,Mint,Lion,Base,Leather Jacket - Black and White,Grin - 5pm Shadow,Eyepatch - Angry,Pylon Helmet,208630217786676767860015082049195109707
...,...,...,...,...,...,...,...,...,...
9995,Terrapin #00146,Charcoal Full Moon,Tiger Teddy,Shell Spikes,Naked,Terrorpin,Steam Punk - Tweeker,Spiky - Black,140850908578020633267983016681440335825
9996,Terrapin #00147,Early Light,Common,Base,Hawaiian Shirt - Pineapple,Moustache,Cyborg - Bored,Lion Hat,198469524447758964403111579286542946696
9997,Terrapin #00148,Midnight Blue - Big Moon,Tiger Teddy,Angel Wings,Naked,Dopey,Angry Tweeker,Lion Hat,7349082768268168848355752264853030978
9998,Terrapin #00149,Mint,Tiger,Base,Naked,Dopey,Eyepatch - Bored,Lion Hat,207748826621535591411786181607822466045


In [21]:
mintNFT_df.to_csv(dataset_path+'mints_data.csv', index=False)